In [44]:
import re
import time
import json
import csv
import random
import tweepy
import requests
import pprint
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import chisquare
from collections import defaultdict

In [45]:
data_root = '/home/haopeng/Data/Retraction/'

In [70]:
# Twitter API (v1) credentials

consumer_key = 'xx'
consumer_secret = 'xx'
access_key = 'xx'
access_secret = 'xx'


auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth, wait_on_rate_limit_notify = True)

In [71]:
api.get_status('952252021733842944', tweet_mode = "extended")

Status(_api=<tweepy.api.API object at 0x7f945f9e6c50>, _json={'created_at': 'Sat Jan 13 18:52:28 +0000 2018', 'id': 952252021733842944, 'id_str': '952252021733842944', 'full_text': "RT @tarlachmc: Let's hope the High Level Group will reach out accordingly &amp; nevertheless show a high level of appreciation of the importanc…", 'truncated': False, 'display_text_range': [0, 144], 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'tarlachmc', 'name': 'Tarlach McGonagle', 'id': 1292531509, 'id_str': '1292531509', 'indices': [3, 13]}], 'urls': []}, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 306379904, 'id_str': '306379904', 'name': 'Irene Roche Laguna', 'screen_name': 'rochelag', 'location': '', 'description': 'Back to online, or digital,

## Get tweets (retract + control)

In [50]:
tweet_ids = set()

with open(data_root+"retraction_altmetric.json", 'r') as ffile:
    for row in ffile:
        paper = json.loads(row)
        if 'posts' in paper:
            if 'twitter' in paper['posts']:
                for tw in paper['posts']['twitter']:
                    tweet_ids.add(tw['tweet_id'])

# Can skip the code below before getting the matched control papers.
with open(data_root+"control_alt_papers.json", 'r') as ffile:
    for row in ffile:
        paper = json.loads(row)
        if 'posts' in paper:
            if 'twitter' in paper['posts']:
                for tw in paper['posts']['twitter']:
                    tweet_ids.add(tw['tweet_id'])

In [51]:
len(tweet_ids)

215986

In [69]:
i = 0
crawl_stat = {}

with open(data_root+'tweets.json', 'a') as fout:
    for tid in tweet_ids:
        i += 1
        try:
            status = api.get_status(tid, tweet_mode = "extended")
            fout.write(json.dumps(status._json)+'\n')
        except tweepy.RateLimitError:
            time.sleep(15 * 60)
        except tweepy.TweepError as e:
            crawl_stat[tid] = str(e)
            pass
        if i%10000 == 0:
            print('processed %d/%d tweets'%(i, len(tweet_ids)))

processed 10000/215986 tweets
processed 20000/215986 tweets
processed 30000/215986 tweets
processed 40000/215986 tweets
processed 50000/215986 tweets
processed 60000/215986 tweets
processed 70000/215986 tweets
processed 80000/215986 tweets
processed 90000/215986 tweets
processed 100000/215986 tweets
processed 110000/215986 tweets
processed 120000/215986 tweets
processed 130000/215986 tweets
processed 140000/215986 tweets
processed 150000/215986 tweets
processed 160000/215986 tweets
processed 170000/215986 tweets
processed 180000/215986 tweets
processed 190000/215986 tweets
processed 200000/215986 tweets
processed 210000/215986 tweets


In [1]:
! wc -l /home/haopeng/Data/Retraction/tweets.json

194643 /home/haopeng/Data/Retraction/tweets.json


In [2]:
194643 / 215986

0.9011834100358357

In [41]:
status = api.get_status('1058127367498010625', tweet_mode = "extended")

In [43]:
# still truncated ...
status._json['full_text']

'RT @Mark_A_Lunn: Why am I not surprised?\n\nIt seems every new study says "faster than we thought" or "worse than we thought."\n\nFor those of…'

## Get all tweets indexed in Altmetric data

In [27]:
# code to read all tids from Alt db is deleted.
# 
# 85411606 tids in total, which contains 14412 tids that are not numeric string.
# e.g., '1.5558470649316E+17', '1.5464891076012E+17', '1.6227338710274E+17'
len(all_tids_alt)

85411606

Collecting process omitted here (using Twitter Academic API)

In [50]:
tid_date_all = {}

with open(data_root+'second_match/tweets_clean.json', 'r') as ifile:
    for line in ifile:
        line = json.loads(line)
        tid, tdate = line['id_str'], line['created_at']
        tid_date_all[tid] = datetime.strptime(tdate, '%a %b %d %H:%M:%S +0000 %Y')#.strftime("%Y-%m-%d")

In [61]:
len(tid_date_all)

77497304